In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import functional
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader,random_split
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
if torch.cuda.is_available():
    print('CUDA is available!  Training on GPU ...')
    device=torch.device('cuda')
else:
    print('CUDA is not available.  Training on CPU ...')
    device=torch.device('cpu')

In [ ]:
path='../input/100-bird-species/birds/'
num_workers = 3
batch_size = 20
valid_size = 0.2
train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
train_data = datasets.ImageFolder(path+'train', transform=train_transform)
test_data = datasets.ImageFolder(path+'test', transform=test_transform)
val_data = datasets.ImageFolder(path+'valid', transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers,shuffle=True)
valid_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

In [ ]:
%matplotlib inline

def imshow(img):
    img = img *0.25 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0))) 
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy()

fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])

In [ ]:
class alex_net(nn.Module):
    def __init__(self):
        super(alex_net,self).__init__()
        self.C1=nn.Conv2d(3,64,kernel_size=(11,11),stride=4,padding=2)
        self.pool=nn.MaxPool2d(3,2)
        self.C2=nn.Conv2d(64,192,kernel_size=(5,5),padding=2)
        self.C3=nn.Conv2d(192,384,kernel_size=(3,3),padding=1)
        self.C4=nn.Conv2d(384,256,kernel_size=(3,3),padding=1)
        self.C5=nn.Conv2d(256,256,kernel_size=(3,3),padding=1)
        self.fc1=nn.Linear(9216,4096)
        self.fc2=nn.Linear(4096,1024)
        self.fc3=nn.Linear(1024,275)
        self.softmax=nn.LogSoftmax(dim=1)
    def forward(self,x):
        x=self.pool(F.relu(self.C1(x)))
        x=self.pool(F.relu(self.C2(x)))
        x=F.relu(self.C3(x))
        x=F.relu(self.C4(x))
        x=self.pool(F.relu(self.C5(x)))
        x=x.view(x.shape[0],-1)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        x=self.softmax(x)
        return x
model=alex_net()
model=model.to(device)
print(model)


In [ ]:
%pip install torch-summary
from torchsummary import summary
summary=summary(model)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.0003)
criterion=nn.NLLLoss()

In [ ]:
from tqdm.notebook import tqdm
n_epochs = 5

last_loss=None
train_losses, val_losses, train_accuracy ,val_accuracy =[] ,[], [], []
for epoch in tqdm(range(1, n_epochs+1)):
    train_class_correct = list(0. for i in range(275))
    train_class_total = list(0. for i in range(275))
    val_class_correct = list(0. for i in range(275))
    val_class_total = list(0. for i in range(275))
    train_loss = 0.0
    valid_loss = 0.0
    model.train()
    for image, target in train_loader:
        image, target = image.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*image.size(0)
        
        _, train_pred = torch.max(output, 1)
        train_correct = np.squeeze(train_pred.eq(target.data.view_as(train_pred)))
        for i in range(len(target)):
            label = target.data[i].item()
            train_class_correct[label] += train_correct[i].item()
            train_class_total[label] += 1
    train_accuracy.append(np.sum(train_class_correct)*100 / np.sum(train_class_total))
    model.eval()
    for image, target in valid_loader:
        image, target = image.to(device), target.to(device)
        output = model(image)
        loss = criterion(output, target)
        valid_loss += loss.item()*image.size(0)
        _, val_pred = torch.max(output, 1)
        val_correct = np.squeeze(val_pred.eq(target.data.view_as(val_pred)))
        for i in range(len(target)):
            label = target.data[i].item()
            val_class_correct[label] += val_correct[i].item()
            val_class_total[label] += 1
    val_accuracy.append(np.sum(val_class_correct)*100 / np.sum(val_class_total))
        
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    train_losses.append(train_loss)
    val_losses.append(valid_loss)

In [ ]:
plt.title("loss vs epochs")
plt.xlabel('Number of epochs')
plt.ylabel('loss')
plt.plot(train_losses,label='train')
plt.plot(val_losses,label='val')
plt.legend()
plt.show()

In [ ]:
plt.title("accuracy vs epochs")
plt.xlabel('Number of epochs')
plt.ylabel('accuracy')
plt.plot(train_accuracy,label='train')
plt.plot(val_accuracy,label='val')
plt.legend()
plt.show()

In [ ]:
test_loss = 0.0
class_correct = list(0. for i in range(275))
class_total = list(0. for i in range(275))
model=model.to(device)
classes=list(range(275))
model.eval()

for image, target in test_loader:
    image, target = image.to(device), target.to(device)
    output = model(image)
    loss = criterion(output, target)
    test_loss += loss.item()*image.size(0)
    _, pred = torch.max(output, 1)
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    for i in range(len(target)):
        label = target.data[i].item()
        class_correct[label] += correct[i].item()
        class_total[label] += 1
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.5f}\n'.format(test_loss))
print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))